# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4945, uptime 0:00:34
mini_beamline                    RUNNING   pid 4946, uptime 0:00:34
random_walk                      RUNNING   pid 4947, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4948, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4949, uptime 0:00:34
simple                           RUNNING   pid 4950, uptime 0:00:34
thermo_sim                       RUNNING   pid 4951, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4952, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-20 15:46:32
Persistent Unique Scan ID: 'fbd2943d-7338-4ae3-ae23-a06bd87832cc'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 15:46:32.6 |     -1.000 |      99684 |


|         2 | 15:46:32.6 |     -0.500 |     103202 |
|         3 | 15:46:32.7 |      0.000 |     103728 |
|         4 | 15:46:32.7 |      0.500 |     103266 |
|         5 | 15:46:32.7 |      1.000 |     101802 |


+-----------+------------+------------+------------+
generator scan ['fbd2943d'] (scan num: 3)





('fbd2943d-7338-4ae3-ae23-a06bd87832cc',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/fbd2943d-7338-4ae3-ae23-a06bd87832cc-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-20 15:46:33
Persistent Unique Scan ID: '09ecadfa-9c8a-4ac0-8043-d11b6f0e7f3b'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 15:46:33.2 |          -1 | 1131735584 |


|         2 | 15:46:33.4 |          -0 | 1130478403 |


|         3 | 15:46:33.5 |           0 | 1127969998 |


|         4 | 15:46:33.6 |           0 | 1123749604 |


|         5 | 15:46:33.7 |           1 | 1118755935 |


+-----------+------------+-------------+------------+
generator scan ['09ecadfa'] (scan num: 4)





('09ecadfa-9c8a-4ac0-8043-d11b6f0e7f3b',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-20 15:46:34
Persistent Unique Scan ID: 'a62350b9-0fb0-46b8-ad39-1a34cad665c5'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 15:46:34.2 | 1096457380 |


|         2 | 15:46:35.2 | 1053236181 |


|         3 | 15:46:36.2 | 1093725628 |


...

|        28 | 15:47:01.2 | 1131774001 |


|        29 | 15:47:02.2 | 1089093675 |


|        30 | 15:47:03.2 | 1053457084 |


+-----------+------------+------------+
generator count ['a62350b9'] (scan num: 5)





('a62350b9-0fb0-46b8-ad39-1a34cad665c5',)